In [ ]:
import numpy as np
from PIL import Image
import random
import math
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

def distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def linear_interpolation(color1, color2, t):
    return (np.array(color1)[:, None] * (1 - t) + np.array(color2)[:, None] * t).T

def sigmoid(x, a=1.0, b=0.0):
    return 1 / (1 + np.exp(-a * (x - b)))

def softstep(x, slope=1.0):
    return (1 / (1 + np.exp(-slope * (x - 0.5))))

def sort_points_by_x(points):
    return sorted(points, key=lambda p: p[0])

def sort_points_by_distance(points):
    if len(points) <= 1:
        return points

    sorted_points = [points.pop(random.randint(0, len(points) - 1))]

    while points:
        last_point = sorted_points[-1]
        distances = [distance(last_point[0], last_point[1], p[0], p[1]) for p in points]
        nearest_index = np.argmin(distances)
        sorted_points.append(points.pop(nearest_index))
    
    return sorted_points

def generate_spline_gradient_image(width, height, points, start_color, end_color, max_distance):
    t_values_points = np.linspace(0, 1, num=len(points))

    points_x = [p[0] for p in points]
    points_y = [p[1] for p in points]

    cx = CubicSpline(t_values_points, points_x)
    cy = CubicSpline(t_values_points, points_y)
    
    data = np.zeros((height, width, 3), dtype=np.uint8)

    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x.flatten()
    y = y.flatten()

    t_values = np.linspace(0, 1, num=500)
    point_distance = point_to_curve_distance(x, y, t_values, cx, cy)
    
    # Apply a nonlinear function based on distance (e.g., softstep)
    normalized_distance = point_distance / max_distance
    softstep_distance = softstep(normalized_distance, slope=5.0)  
    ratio = np.clip(softstep_distance, 0, 1)
    
    colors = linear_interpolation(start_color, end_color, ratio).astype(np.uint8)
    data = colors.reshape((height, width, 3))

    img = Image.fromarray(data)
    
    return img, cx, cy

def plot_curve(cx, cy, points):
    t_values = np.linspace(0, 1, num=500)
    curve_x = cx(t_values)
    curve_y = cy(t_values)

    plt.scatter(*zip(*points), color='red')
    plt.plot(curve_x, curve_y, color='blue')

# image setting
width, height = 800, 600
points = [(random.randint(0, width), random.randint(0, height)) for _ in range(2)]
points = [(0, random.randint(0, height))] + points + [(width, random.randint(0, height))]
sorted_points = sort_points_by_x(points.copy())

# color setting
start_color = tuple(random.randint(190, 250) for _ in range(3))
end_color = tuple(random.randint(190, 250) for _ in range(3))

# Parameters for gradient effects
max_distance = 600

# main
spline_gradient_image, cx, cy = generate_spline_gradient_image(width, height, sorted_points, start_color, end_color, max_distance)

# visualization
plt.imshow(spline_gradient_image)
# plot_curve(cx, cy, sorted_points)  # curve checking
plt.axis('off')
plt.show()

# save
spline_gradient_image.save("output.png")